In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

##충돌 방지


In [ ]:
!pip -q uninstall -y torchaudio

!pip -q install --upgrade pip
!pip -q install "torch==2.3.1+cu121" "torchvision==0.18.1+cu121" --index-url https://download.pytorch.org/whl/cu121

!pip -q install "transformers>=4.43.3" "accelerate>=0.33.0" "peft>=0.12.0" "datasets" "einops" "pillow"
!pip -q install "bitsandbytes==0.43.3"
!pip -q install qwen-vl-utils
!pip -q install --upgrade "transformers>=4.45.2" "accelerate>=0.34.2" "qwen-vl-utils>=0.0.8"
!pip install -q triton
!pip install -q bitsandbytes
!pip -q install "triton==2.2.0"
!pip -q install -U transformers accelerate peft pillow einops
!pip -q install -U bitsandbytes

!pip -q uninstall -y torch torchvision torchaudio triton xformers \
  nvidia-cublas-cu12 nvidia-cudnn-cu12 nvidia-cuda-nvrtc-cu12 \
  nvidia-cuda-runtime-cu12 nvidia-cuda-cupti-cu12 nvidia-cusparse-cu12

!pip -q install --index-url https://download.pytorch.org/whl/cu121 \
  torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1

import torch
print("torch:", torch.__version__, "| CUDA:", torch.version.cuda)

!pip -q install -U "transformers>=4.46.0" "accelerate>=0.33.0" "peft>=0.11.1" pillow einops



##멀티모달 학습 및 추론 코드

In [ ]:
import os, io, time, random, math, gc, re, string, hashlib, warnings, logging, json, base64
import torch, pandas as pd, numpy as np
from PIL import Image, ImageFile, ImageOps, ImageFilter
import requests
from requests.adapters import HTTPAdapter, Retry
from urllib.parse import urlparse
from torch.utils.data import Dataset as TorchDataset
from huggingface_hub import login
from transformers import (
    AutoProcessor,
    AutoModelForVision2Seq,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
    TrainerCallback,
    StoppingCriteria,
    StoppingCriteriaList
)
from transformers.trainer_utils import get_last_checkpoint
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from safetensors.torch import load_file as safe_load_file

# ---- (콘솔 묵음 유지) ----
os.environ.setdefault("TRANSFORMERS_VERBOSITY", "error")
logging.getLogger("transformers.generation.utils").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning, module="torch.utils.checkpoint")
Image.MAX_IMAGE_PIXELS = 300_000_000
ImageFile.LOAD_TRUNCATED_IMAGES = True

GLOBAL_START = time.time()

# -----------------------------
# 🧷 사용자 조정 지점
# -----------------------------
MODEL_ID    = "Qwen/Qwen2-VL-7B-Instruct"
CSV_TRAIN   = "/content/drive/MyDrive/deeplearningchallenge/deep_chal_multitask_dataset.parquet"
CSV_TEST    = "/content/drive/MyDrive/deeplearningchallenge/deep_chal_multitask_dataset_test.parquet"

SAVE_DIR    = "/content/drive/MyDrive/aju_ver17/outputs_lora_total"
CKPT_DIR    = os.path.join(SAVE_DIR, "checkpoints")
FINAL_DIR   = SAVE_DIR
SUBMIT_OUT  = "/content/drive/MyDrive/aju_ver17/submission.csv"
SUBMIT_PART = SUBMIT_OUT + ".partial"
IMG_CACHE_DIR   = os.path.join(SAVE_DIR, "img_cache")
SUBSET_IDX_PATH = os.path.join(SAVE_DIR, "train_subset_indices.csv")

# Toggle: 학습/추론
RUN_TRAIN = True

# 부분 학습 서브셋
USE_TRAIN_SUBSET = True
SUBSET_SIZE      = 58000
SUBSET_SEED      = 42
SUBSET_GROUP_CANDIDATES = ["input_type", "task", "lang"]

# 내부 분할
VAL_FRACTION   = 0.10
VAL_GROUP_CANDIDATES = ["type", "task", "category", "label", "input_type", "lang"]

# 시간/저장
LIMIT_HOURS          = 23
AUTOSAVE_EVERY_MIN   = 30
INFER_SAVE_EVERY     = 100

# 메모리/속도
USE_4BIT        = True
MAX_TEXT_TOKENS = 1024

# 🔁 이미지 단일 스펙
MAX_IMAGE_EDGE  = 896
DOC_MAX_EDGE    = 896
DOC_PATCH       = 14
APPLY_DOC_ENHANCE_HTTP = True

LORA_R          = 32
GRAD_ACC_STEPS  = 16
PER_DEV_TRAIN_BS= 1
PER_DEV_EVAL_BS = 1
MAX_GRAD_NORM   = 1.0

# -----------------------------
# ⚡️속도/중복 관련 토글
# -----------------------------
EVAL_EVERY_STEPS        = 400   # ← 400으로 고정
ES_PATIENCE             = 2     # ← 더 빠른 얼리스탑
ES_MIN_DELTA            = 1e-6
EVAL_SAMPLES_PER_TASK   = 40    # 태스크별 샘플 수
EVAL_MAX_TOTAL_SAMPLES  = 200   # 전체 상한
MAX_NEW_TOKENS_EVAL     = 96    # 평가 기본
MAX_NEW_TOKENS_TEST     = 128   # 추론 기본

# 🔎 (이전 호환) 요약 전용 디코딩 상한(미사용 가능)
MAX_NEW_TOKENS_EVAL_SUM = 144
MAX_NEW_TOKENS_TEST_SUM = 196

NO_REPEAT_NGRAM_SIZE    = 4
REPETITION_PENALTY      = 1.05

# ================== 규칙 토글 ==================
UNKNOWN_POLICY        = "off"     # Unknown 완전 차단
CITING_ENFORCEMENT    = "off"
ANTI_REPEAT_POLICY    = "off"     # ← 평가 중 재생성 루프 해제
VQA_SHORTEN_ONE_SENTENCE = True

# 평가/추론 전용 빠른 모드
EVAL_GREEDY_ONLY           = True
EVAL_SKIP_IMAGE_DOWNLOAD   = True   # 평가 시 이미지 로딩 스킵(속도↑)
INFER_GREEDY_ONLY          = True
INFER_SKIP_IMAGE_DOWNLOAD  = False  # 제출 정확도 위해 기본 False 유지

# =================================================
# ✅ 단일 시스템 프롬프트
SYSTEM_INST = (
"""You are a multimodal assistant. Follow the instructions precisely for each task type based on the input.

### Task Definitions & Output Formats ###
# 1. Image Captioning: Image is provided, but no question. -> Output: 3–5 sentences (not fewer than 3, not more than 5).
# 2. Visual Question Answering (VQA): Both an image and a question are provided. -> Output: one concise sentence.
# 3. Math Problem: Only text is provided, the 'question' field is empty, and the text ends with a '?'. -> Output: 1–3 short lines of reasoning, ending with "#### {answer}".
# 4. Text Summarization: Only text is provided, no question, and it does not end with a '?'. -> Output: a coherent paragraph, up to 6 sentences (avoid bullet points and chatty tone).
# 5. Text Question Answering (Text QA): Both text and a question are provided. -> Output: a direct answer or a JSON string if explicitly requested.

Formatting: plain text unless strictly-JSON is explicitly requested. Do not include external knowledge beyond the given inputs.
"""
)

DEFAULT_ASK_IMAGE = "Describe the image in detail, in 3 to 5 sentences."
DEFAULT_ASK_TEXT  = "Summarize the key points of the text in a concise paragraph (up to 6 sentences)."
DEFAULT_ASK_MATH  = "Solve the math problem, showing 1–4 compact steps, with the final answer prefixed by '#### ' on the last line."

# 🔒 금칙어(디코딩 후 제거)
BANNED_PHRASES = [
    "I'm sorry", "I am sorry", "I apologize", "Apologies",
    "As an AI", "as a language model",
    "I don't have", "I do not have",
    "I cannot", "I can't", "cannot provide", "can't provide",
    "I have no information", "no information about",
    "I cannot answer", "I can't answer", "Hello! How can I assist you today?"
]

# 토큰 (선택)
token = os.getenv("HF_TOKEN")
if token:
    login(token=token, add_to_git_credential=True)
    os.environ["HUGGINGFACE_HUB_TOKEN"] = token

# 재현성
torch.manual_seed(SUBSET_SEED); random.seed(SUBSET_SEED); np.random.seed(SUBSET_SEED)
print("torch:", torch.__version__, "| CUDA:", torch.version.cuda)

# 성능/안정화
torch.backends.cuda.matmul.allow_tf32 = True
USE_BF16 = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8
USE_FP16 = not USE_BF16 and torch.cuda.is_available()

# -----------------------------
# 🔢 Per-task 토큰 예산 & 문장 상한
# -----------------------------
CAPTION_MAX_NEW_TOKENS_EVAL = MAX_NEW_TOKENS_EVAL * 2
CAPTION_MAX_NEW_TOKENS_TEST = MAX_NEW_TOKENS_TEST * 2
CAPTION_MAX_SENTENCES = 5

MATH_MAX_NEW_TOKENS_EVAL = MAX_NEW_TOKENS_EVAL * 2
MATH_MAX_NEW_TOKENS_TEST = MAX_NEW_TOKENS_TEST * 2

SUM_MAX_NEW_TOKENS_EVAL = MAX_NEW_TOKENS_EVAL * 4
SUM_MAX_NEW_TOKENS_TEST = MAX_NEW_TOKENS_TEST * 4
SUM_MAX_SENTENCES = 6

# -----------------------------
# 유틸
# -----------------------------
def safe_read_df(path):
    try:
        if path.endswith(".parquet"): return pd.read_parquet(path)
        if path.endswith(".csv"):     return pd.read_csv(path)
        try:                          return pd.read_parquet(path)
        except Exception:             return pd.read_csv(path)
    except Exception as e:
        raise RuntimeError(f"파일 읽기 실패: {path} | {e}")

def _round_to_multiple(x, m): return max(m, int(round(x/m))*m)

def _cache_path(url: str, max_edge: int):
    os.makedirs(IMG_CACHE_DIR, exist_ok=True)
    h = hashlib.sha1(f"{url}|{max_edge}".encode()).hexdigest()
    return os.path.join(IMG_CACHE_DIR, f"{h}.jpg")

# ===== 문서 이미지 보정 =====
def _resize_to_grid(img: Image.Image, max_edge=DOC_MAX_EDGE, patch=DOC_PATCH):
    w, h = img.size
    if max(w, h) > max_edge:
        scale = max_edge / float(max(w, h))
        nw = max(patch, _round_to_multiple(int(w * scale), patch))
        nh = max(patch, _round_to_multiple(int(h * scale), patch))
    else:
        nw = _round_to_multiple(w, patch)
        nh = _round_to_multiple(h, patch)
    return img.resize((nw, nh), Image.Resampling.LANCZOS)

def enhance_doc(img: Image.Image) -> Image.Image:
    g = ImageOps.grayscale(img)
    g = ImageOps.autocontrast(g)
    g = g.filter(ImageFilter.UnsharpMask(radius=2, percent=150, threshold=3))
    return g.convert("RGB")

def is_base64_like(s: str) -> bool:
    if not isinstance(s, str): return False
    if s.startswith("data:image/"): return True
    return bool(re.match(r"^[A-Za-z0-9+/=\s]{50,}$", s))

def load_image_raw_base64(inp: str):
    try:
        data = inp.split(",", 1)[1] if inp.startswith("data:image/") else inp
        raw  = base64.b64decode(data, validate=False)
        return Image.open(io.BytesIO(raw)).convert("RGB")
    except Exception:
        return None

def make_doc_views_from_base64(inp: str):
    if not is_base64_like(inp): return None
    img = load_image_raw_base64(inp)
    if img is None: return None
    base = enhance_doc(img)
    full = _resize_to_grid(base, DOC_MAX_EDGE, DOC_PATCH)
    return [full]

# =========================
# ⚙️ 강건한 HTTP 로더 (재시도↓, 타임아웃↓)
# =========================
_RETRY = Retry(
    total=1, backoff_factor=0.4,
    status_forcelist=(429,500,502,503,504),
    allowed_methods=frozenset(["GET","HEAD"])
)
_SESSION = requests.Session()
_SESSION.mount("https://", HTTPAdapter(max_retries=_RETRY))
_SESSION.mount("http://",  HTTPAdapter(max_retries=_RETRY))
_DEFAULT_HEADERS = {
    "User-Agent": ("Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                   "(KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"),
    "Accept": "image/webp,image/*;q=0.8,*/*;q=0.5",
    "Accept-Language": "en-US,en;q=0.9,ko;q=0.8",
}
def _origin(url: str) -> str:
    u = urlparse(url)
    return f"{u.scheme}://{u.netloc}"
def _looks_like_image_bytes(head: bytes) -> bool:
    return head.startswith(b"\xff\xd8") or head.startswith(b"\x89PNG") or head.startswith(b"RIFF")

def load_image_resized(inp, max_edge=MAX_IMAGE_EDGE, patch=14, timeout_s=8):
    if isinstance(inp, str) and inp.startswith(("http://","https://")):
        path=_cache_path(inp, max_edge)
        if os.path.exists(path):
            try: return Image.open(path).convert("RGB")
            except Exception: pass
        headers = dict(_DEFAULT_HEADERS); headers["Referer"] = _origin(inp)
        try:
            resp = _SESSION.get(inp, headers=headers, timeout=timeout_s, allow_redirects=True, stream=True)
            if resp.status_code != 200:
                print(f"[HTTP] status={resp.status_code} url={inp}")
                return None
            ctype = (resp.headers.get("Content-Type") or "").lower()
            content = resp.content
            if "image" not in ctype and not _looks_like_image_bytes(content[:16]):
                print(f"[HTTP] non-image content-type={ctype} url={inp}")
                return None
            img = Image.open(io.BytesIO(content))
            if img.mode not in ("RGB","RGBA"):
                img = img.convert("RGB")
            elif img.mode == "RGBA":
                img = img.convert("RGB")
            if APPLY_DOC_ENHANCE_HTTP:
                try: img = enhance_doc(img)
                except Exception: pass
        except Exception as e:
            print(f"[HTTP] fetch failed: {e} url={inp}")
            return None

        w,h=img.size
        if max(w,h)>max_edge:
            scale=max_edge/float(max(w, h))
            new_w=_round_to_multiple(int(w*scale),patch)
            new_h=_round_to_multiple(int(h*scale),patch)
            img=img.resize((max(patch,new_w),max(patch,new_h)), Image.Resampling.LANCZOS)
        else:
            img=img.resize((_round_to_multiple(w,patch),_round_to_multiple(h,patch)), Image.Resampling.LANCZOS)
        try: img.save(path, format="JPEG", quality=92)
        except Exception: pass
        return img

    # base64
    if isinstance(inp, str):
        try:
            data = inp.split(",", 1)[1] if inp.startswith("data:image/") else inp
            if re.match(r"^[A-Za-z0-9+/=\s]{100,}$", data or ""):
                raw = base64.b64decode(data, validate=False)
                img = Image.open(io.BytesIO(raw)).convert("RGB")
                w, h = img.size
                if max(w, h) > max_edge:
                    scale = max_edge / float(max(w, h))
                    new_w = max(patch, _round_to_multiple(int(w * scale), patch))
                    new_h = max(patch, _round_to_multiple(int(h * scale), patch))
                    img = img.resize((new_w, new_h), Image.Resampling.LANCZOS)
                else:
                    img = img.resize((_round_to_multiple(w, patch), _round_to_multiple(h, patch)), Image.Resampling.LANCZOS)
                return img
        except Exception:
            pass
    return None

# -----------------------------
# ✅ 태스크 라우팅(라벨 미사용, 결정 규칙만)
# -----------------------------
def detect_input_kind(s: str) -> str:
    if not isinstance(s, str): return "unknown"
    s = s.strip()
    if s.startswith(("http://","https://")): return "image"
    if re.match(r"^data:image/", s) or re.match(r"^[A-Za-z0-9+/=\s]{100,}$", s or ""):
        return "image"
    return "text"

def determine_task(input_type: str, context: str, question: str) -> str:
    it = (input_type or "").strip().lower()
    ctx = (context or "").strip()
    q  = ("" if question is None else str(question)).strip()

    if it not in ("image","text"):
        it = detect_input_kind(context)

    if it == "image":
        return "vqa" if q else "captioning"

    if it == "text":
        if q: return "text_qa"
        if ctx.endswith("?"): return "math" 
        return "summarization"

    return "summarization"

# -----------------------------
# 입력 기반 프롬프트
# -----------------------------
def build_text_prompt(input_type: str, context: str, question: str):
    itype = (input_type or "").strip().lower()
    ctx = (context or "").strip()
    q   = ("" if question is None else str(question)).strip()

    task_r = determine_task(itype, ctx, q)

    parts = []
    if itype == "text" and ctx:
        parts.append(ctx)

    if q:
        parts.append("Question: " + q)
    else:
        if task_r == "captioning":
            parts.append("Question: " + DEFAULT_ASK_IMAGE)
        elif task_r == "math":
            parts.append("Question: " + DEFAULT_ASK_MATH)
        else:
            parts.append("Question: " + DEFAULT_ASK_TEXT)

    if itype == "image":
        if q:
            parts.append("Constraints: Provide one concise sentence grounded only in the image.")
        else:
            parts.append("Constraints: Provide 3–5 sentences, grounded only in the image.")

    return "\n".join(parts).strip()

# -----------------------------
# 후처리(형식 강제 + 빈 출력 가드 "-")
# -----------------------------
def _first_sentence(s: str) -> str:
    m = re.split(r"(?<=[.!?])\s+", s.strip(), maxsplit=1)
    return m[0] if m and m[0] else s.strip()

def _strip_bullets_and_chatty(s: str) -> str:
    lines = [re.sub(r'^\s*(?:[-*•]+|\d+\.)\s*', '', ln) for ln in s.splitlines()]
    s2 = " ".join([ln.strip() for ln in lines if ln.strip()])
    s2 = re.sub(r"[👍✨💡✅❗️❗️🤔📝]+", "", s2)
    return re.sub(r"\s+", " ", s2).strip()

def postprocess_output(ans: str,
                       question: str = None,
                       input_type: str = None,
                       image_loaded: bool = True,
                       context: str = None):
    s = (ans or "")

    for p in BANNED_PHRASES:
        s = s.replace(p, "")
    s = re.sub(r"\s+", " ", s).strip()

    if UNKNOWN_POLICY == "off":
        if re.fullmatch(r"(?i)\s*unknown\s*", s): s = ""
        s = re.sub(r"(?i)\bunknown\b", "", s).strip()

    s = re.sub(r"(?<=\d),(?=\d)", "", s)

    it = (input_type or "").strip().lower()
    q  = (question or "").strip()
    ctx = (context or "").strip()
    task_r = determine_task(it, ctx, q)

    if task_r == "math" and it != "image":
        if "####" not in s:
            nums = re.findall(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?", s)
            tail = nums[-1] if nums else (s.split()[-1] if s else "")
            s = f"{s}\n#### {tail}".strip()

    if it == "image":
        s = re.sub(r"\s*####\s*.*$", "", s).strip()
        if q and VQA_SHORTEN_ONE_SENTENCE:
            s = _first_sentence(s)

    if task_r == "summarization":
        s = _strip_bullets_and_chatty(s)

    if not s.strip():
        s = "-"

    return s

# -----------------------------
# ⛔ 문장 개수 상한 스톱 기준
# -----------------------------
class StopOnSentenceCount(StoppingCriteria):
    def __init__(self, tokenizer, start_pos: int, max_sentences: int):
        self.tok = tokenizer
        self.start_pos = int(start_pos)
        self.max_sentences = int(max_sentences)
        self._pat = re.compile(r'[.!?](?=\s|$|["”])')
    def __call__(self, input_ids, scores, **kwargs) -> bool:
        new_ids = input_ids[0][self.start_pos:]
        if new_ids.numel() <= 0:
            return False
        text = self.tok.decode(new_ids, skip_special_tokens=True)
        ends = self._pat.findall(text)
        return len(ends) >= self.max_sentences

# -----------------------------
# 모델/프로세서 (QLoRA 4bit)
# -----------------------------
bnb = None
if USE_4BIT:
    try:
        bnb = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16 if USE_BF16 else torch.float16,
        )
    except Exception as e:
        print("bitsandbytes 설정 실패 → 4bit 비활성:", e)
        USE_4BIT = False
        bnb = None

device_map = "auto" if torch.cuda.is_available() else "cpu"
model = AutoModelForVision2Seq.from_pretrained(
    MODEL_ID,
    device_map=device_map,
    quantization_config=bnb,
    trust_remote_code=True,
    token=token
)
processor = AutoProcessor.from_pretrained(MODEL_ID, trust_remote_code=True, token=token)

model = prepare_model_for_kbit_training(model)

# 비전 타워 동결
for n, p in model.named_parameters():
    if any(k in n for k in ["vision","vision_tower","clip","image","vision_model"]):
        p.requires_grad = False

# LoRA
lora = LoraConfig(
    r=LORA_R, lora_alpha=16, lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    bias="none", task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora)

if hasattr(model.config, "use_cache"):
    model.config.use_cache = False  # gradient_checkpointing 호환

# trainable % 출력
def count_trainable(m):
    t=a=0
    for p in m.parameters():
        a+=p.numel()
        if p.requires_grad: t+=p.numel()
    return t,a
t,a = count_trainable(model)
print(f"trainable params: {t:,} || all params: {a:,} || trainable%: {t/a*100:.4f}")
print("모델 + LoRA 준비 완료")

# -----------------------------
# 데이터 로드(+클린) → 서브셋 → 90/10 분할
# -----------------------------
df_full = safe_read_df(CSV_TRAIN).copy()
for col in ["input_type","input","question","output","task"]:
    if col not in df_full.columns:
        df_full[col] = "" if col!="task" else "unknown"
df_full = df_full[~df_full["output"].isna()]
df_full = df_full[df_full["output"].astype(str).str.strip().str.len()>0].reset_index(drop=True)
df_full["_orig_idx"] = np.arange(len(df_full))

def stratified_or_random_sample(df, n=8000, by_cols=None, seed=42):
    n=min(n,len(df))
    if n<=0: return df.head(0).copy()
    if by_cols:
        tot=len(df); parts=[]; remain=n
        for _,g in df.groupby(by_cols, dropna=False, group_keys=False):
            alloc=max(1,int(round(len(g)/tot*n))); take=min(alloc,len(g))
            parts.append(g.sample(n=take, random_state=seed)); remain-=take
        out=pd.concat(parts) if parts else df.head(0)
        if remain>0:
            rest=df.drop(out.index, errors="ignore")
            if len(rest)>0: out=pd.concat([out, rest.sample(n=min(remain,len(rest)), random_state=seed)])
        if len(out)>n: out=out.sample(n=n, random_state=seed)
        return out.sample(frac=1.0, random_state=seed).reset_index(drop=True)
    else:
        return df.sample(n=n, random_state=seed).reset_index(drop=True)

def stratified_train_val_split(df, val_frac=0.1, by_cols=None, seed=42):
    df=df.sample(frac=1.0, random_state=seed).reset_index(drop=True)
    if not by_cols:
        n_val=max(1,int(round(len(df)*val_frac)))
        return df.iloc[n_val:].reset_index(drop=True), df.iloc[:n_val].reset_index(drop=True)
    parts_tr,parts_val=[],[]
    for _,g in df.groupby(by_cols, dropna=False, group_keys=False):
        if len(g)==1: parts_tr.append(g); continue
        n_val=max(1,int(round(len(g)*val_frac))); g=g.sample(frac=1.0, random_state=seed)
        parts_val.append(g.iloc[:n_val]); parts_tr.append(g.iloc[n_val:])
    tr=pd.concat(parts_tr).sample(frac=1.0, random_state=seed).reset_index(drop=True)
    va=pd.concat(parts_val).sample(frac=1.0, random_state=seed).reset_index(drop=True)
    return tr, va

if USE_TRAIN_SUBSET:
    os.makedirs(SAVE_DIR, exist_ok=True)
    if os.path.exists(SUBSET_IDX_PATH):
        try:
            idx_df = pd.read_csv(SUBSET_IDX_PATH)
            keep_idx = idx_df["_orig_idx"].astype(int).tolist()
            df_core = df_full.iloc[keep_idx].reset_index(drop=True)
            print(f"★ 부분 학습 모드(재개): subset size = {len(df_core)} (from {len(df_full)})")
        except Exception as e:
            print("부분 인덱스 로드 실패, 새로 샘플링:", e)
            by_cols=[c for c in SUBSET_GROUP_CANDIDATES if c in df_full.columns]
            df_core = stratified_or_random_sample(df_full, n=SUBSET_SIZE, by_cols=by_cols, seed=SUBSET_SEED)
            pd.DataFrame({"_orig_idx": df_core["_orig_idx"].tolist()}).to_csv(SUBSET_IDX_PATH, index=False)
            print(f"★ 부분 학습 모드(신규): subset size = {len(df_core)} → 인덱스 저장")
    else:
        by_cols=[c for c in SUBSET_GROUP_CANDIDATES if c in df_full.columns]
        df_core = stratified_or_random_sample(df_full, n=SUBSET_SIZE, by_cols=by_cols, seed=SUBSET_SEED)
        pd.DataFrame({"_orig_idx": df_core["_orig_idx"].tolist()}).to_csv(SUBSET_IDX_PATH, index=False)
        print(f"★ 부분 학습 모드(신규): subset size = {len(df_core)} → 인덱스 저장")
else:
    df_core = df_full.reset_index(drop=True)

by_cols_for_split=[c for c in VAL_GROUP_CANDIDATES if c in df_core.columns]
print("층화 분할 기준:", by_cols_for_split if by_cols_for_split else "없음(랜덤)")

df_tr, df_va = stratified_train_val_split(df_core, val_frac=VAL_FRACTION, by_cols=by_cols_for_split, seed=SUBSET_SEED)

pd.DataFrame({"_orig_idx": df_tr["_orig_idx"]}).to_csv(os.path.join(SAVE_DIR, "train_split_indices.csv"), index=False)
pd.DataFrame({"_orig_idx": df_va["_orig_idx"]}).to_csv(os.path.join(SAVE_DIR, "val_split_indices.csv"), index=False)
for x in (df_tr, df_va):
    if "_orig_idx" in x.columns: x.drop(columns=["_orig_idx"], inplace=True)

print(f"final train size: {len(df_tr)} | final val size: {len(df_va)}")

# -----------------------------
# 데이터셋/콜레이트
# -----------------------------
class MultiModalDataset(TorchDataset):
    def __init__(self, df, processor):
        self.df=df.reset_index(drop=True)
        self.processor=processor
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row=self.df.iloc[idx]
        input_type=row.get("input_type","")
        question=row.get("question","")
        context=row.get("input","")
        answer=row.get("output","")

        user_text = build_text_prompt(input_type, context, question)
        # 학습 시에는 이미지 로딩 유지(정확도)
        image = load_image_resized(context) if input_type=="image" else None

        if image is not None and input_type=="image":
            messages=[
                {"role":"system","content":[{"type":"text","text":SYSTEM_INST}]},
                {"role":"user","content":[{"type":"image"},{"type":"text","text":user_text or ""}]}
            ]; images=[image]
        else:
            messages=[
                {"role":"system","content":[{"type":"text","text":SYSTEM_INST}]},
                {"role":"user","content":[{"type":"text","text":user_text or ""}]}
            ]; images=None

        prompt=self.processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        enc=self.processor(text=prompt, images=images, return_tensors="pt", padding=False)

        # ✅ 학습 토큰 클립: 프롬프트 보존, 정답만 예산 내
        enc_ids = enc["input_ids"].squeeze(0)
        enc_att = enc["attention_mask"].squeeze(0)
        ans_ids = self.processor.tokenizer(
            str(answer) if isinstance(answer,str) else "",
            return_tensors="pt", add_special_tokens=True, padding=False
        ).input_ids.squeeze(0)
        allow = max(0, MAX_TEXT_TOKENS - enc_ids.size(0))
        if ans_ids.size(0) > allow:
            ans_ids = ans_ids[:allow]

        in_ids  = torch.cat([enc_ids, ans_ids], dim=0)
        attn    = torch.cat([enc_att, torch.ones_like(ans_ids)], dim=0)

        prompt_len = enc_ids.size(0)
        labels = in_ids.clone()
        labels[:prompt_len] = -100

        item={"input_ids":in_ids.long(),"attention_mask":attn.long(),"labels":labels.long()}
        if "pixel_values" in enc: item["pixel_values"]=enc["pixel_values"].squeeze(0)
        if "image_grid_thw" in enc: item["image_grid_thw"]=enc["image_grid_thw"].squeeze(0)
        return item

def collate(batch):
    pad_id = processor.tokenizer.pad_token_id or 0
    def pad_1d(tensors, pad_val, dtype=torch.long):
        m = max(t.size(0) for t in tensors); out=[]
        for t in tensors:
            if t.size(0)<m:
                pad=torch.full((m-t.size(0),), pad_val, dtype=t.dtype)
                out.append(torch.cat([t,pad], dim=0))
            else: out.append(t)
        return torch.stack(out, dim=0).to(dtype)
    b={}
    b["input_ids"]      = pad_1d([x["input_ids"] for x in batch], pad_id, torch.long)
    b["attention_mask"] = pad_1d([x["attention_mask"] for x in batch], 0, torch.long)
    b["labels"]         = pad_1d([x["labels"] for x in batch], -100, torch.long)
    if all("pixel_values" in x for x in batch):
        b["pixel_values"]=torch.stack([x["pixel_values"] for x in batch], dim=0)
    if all("image_grid_thw" in x for x in batch):
        grids=[x["image_grid_thw"] for x in batch]
        try: b["image_grid_thw"]=torch.stack(grids, dim=0)
        except Exception: b["image_grid_thw"]=grids
    return b

train_ds = MultiModalDataset(df_tr, processor)
val_ds   = MultiModalDataset(df_va, processor)

# -----------------------------
# 커스텀 지표(라우팅 기반)
# -----------------------------
_punc_tbl = str.maketrans("", "", string.punctuation)
def _normalize(s):
    s = ("" if s is None else str(s)).strip().lower()
    s = s.translate(_punc_tbl)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def exact_match(pred, ref): return 1.0 if _normalize(pred) == _normalize(ref) else 0.0

def token_f1(pred, ref):
    p = _normalize(pred).split(); r = _normalize(ref).split()
    if len(p)==0 and len(r)==0: return 1.0
    if len(p)==0 or len(r)==0:  return 0.0
    from collections import Counter
    pc, rc = Counter(p), Counter(r)
    common = sum((pc & rc).values())
    if common==0: return 0.0
    prec = common / max(1, len(p)); rec  = common / max(1, len(r))
    return (2*prec*rec)/(prec+rec) if (prec+rec)>0 else 0.0

def lcs_len(a, b):
    na, nb = len(a), len(b)
    dp = [[0]*(nb+1) for _ in range(na+1)]
    for i in range(1, na+1):
        ai=a[i-1]; row=dp[i]; prow=dp[i-1]
        for j in range(1, nb+1):
            row[j]=prow[j-1]+1 if ai==b[j-1] else max(prow[j], row[j-1])
    return dp[na][nb]

def rouge_l_f1(pred, ref):
    p = _normalize(pred).split(); r = _normalize(ref).split()
    if len(p)==0 and len(r)==0: return 1.0
    if len(p)==0 or len(r)==0:  return 0.0
    l = lcs_len(p, r); prec = l / max(1, len(p)); rec  = l / max(1, len(r))
    return (2*prec*rec)/(prec+rec) if (prec+rec)>0 else 0.0

def numeric_em(pred, ref, tol=1e-3):
    def _num(s):
        m = re.findall(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?", s.replace(",", ""))
        return m[0] if m else None
    pn, rn = _num(pred), _num(ref)
    if pn is None or rn is None: return exact_match(pred, ref)
    try:
        pn, rn = float(pn), float(rn)
        if rn==0: return 1.0 if abs(pn-rn)<=tol else 0.0
        return 1.0 if abs(pn-rn)/max(1e-12, abs(rn)) <= 1e-3 or abs(pn-rn)<=tol else 0.0
    except: return exact_match(pred, ref)

def score_sample_routed(task_r, pred, ref):
    t = (task_r or "").strip().lower()
    if t in ["text_qa", "vqa"]:
        return {"em": exact_match(pred, ref), "f1": token_f1(pred, ref)}
    elif t in ["summarization", "captioning"]:
        return {"rougeL": rouge_l_f1(pred, ref)}
    elif t in ["math"]:
        return {"em": numeric_em(pred, ref)}
    else:
        return {"f1": token_f1(pred, ref)}

def collapse_task_metric(task_r, d):
    t = (task_r or "").strip().lower()
    if t in ["text_qa", "vqa"]:
        return d.get("f1", d.get("em", 0.0))
    elif t in ["summarization", "captioning"]:
        return d.get("rougeL", 0.0)
    elif t in ["math"]:
        return d.get("em", 0.0)
    else:
        return d.get("f1", 0.0)

# -----------------------------
# 평가 서브셋 구성(태스크별 균형 샘플링)
# -----------------------------
def build_eval_subset(df_va, per_task=EVAL_SAMPLES_PER_TASK, max_total=EVAL_MAX_TOTAL_SAMPLES, seed=SUBSET_SEED):
    if len(df_va)==0: return df_va
    tmp = df_va.copy()
    tmp["_routed_task"] = tmp.apply(lambda r: determine_task(r.get("input_type",""), r.get("input",""), r.get("question","")), axis=1)
    parts=[]
    rng = np.random.RandomState(seed)
    for t, g in tmp.groupby("_routed_task", dropna=False):
        take = min(per_task, len(g))
        parts.append(g.sample(n=take, random_state=rng))
    out = pd.concat(parts).sample(frac=1.0, random_state=rng).reset_index(drop=True)
    if len(out) > max_total:
        out = out.sample(n=max_total, random_state=rng).reset_index(drop=True)
    return out.drop(columns=["_routed_task"], errors="ignore")

# -----------------------------
# 시간 제한 + 주기 저장
# -----------------------------
class TimeLimitCallback(TrainerCallback):
    def __init__(self, max_seconds, autosave_every_sec=AUTOSAVE_EVERY_MIN*60):
        self.start = time.time()
        self.max_seconds = max_seconds
        self.autosave_every_sec = autosave_every_sec
        self.last_save = self.start
    def on_step_end(self, args, state, control, **kwargs):
        now=time.time()
        if now - self.last_save >= self.autosave_every_sec:
            control.should_save = True
            self.last_save = now
        if now - self.start >= self.max_seconds:
            print("\n⏰ 시간 제한 도달: 체크포인트 저장 후 학습 종료합니다.")
            control.should_save = True
            control.should_training_stop = True
        return control

# -----------------------------
# 단순 생성(평가/추론용, dedup/샘플링 없음)
# -----------------------------
def _generate_once_greedy(model, enc, max_new_tokens: int, stopping_criteria=None, eos_token_id=None, pad_token_id=None):
    return model.generate(
        **enc,
        do_sample=False,
        num_beams=1,
        max_new_tokens=max_new_tokens,
        no_repeat_ngram_size=NO_REPEAT_NGRAM_SIZE,
        repetition_penalty=REPETITION_PENALTY,
        stopping_criteria=stopping_criteria,
        eos_token_id=eos_token_id,
        pad_token_id=pad_token_id if pad_token_id is not None else eos_token_id
    )

def _decode_new_tokens(processor, gen_ids, enc):
    new_ids = gen_ids[0][enc["input_ids"].shape[-1]:]
    return processor.tokenizer.decode(new_ids, skip_special_tokens=True).strip()

def _fast_generate_row(model, processor, row, stage_name: str, *_args_ignored):
    input_type=row.get("input_type",""); question=row.get("question",""); context=row.get("input","")
    user_text = build_text_prompt(input_type, context, question)
    routed = determine_task(input_type, context, question)

    # 평가/추론에서 이미지 스킵 옵션 반영
    use_images = False
    images = None
    if input_type=="image":
        if stage_name=="eval" and EVAL_SKIP_IMAGE_DOWNLOAD:
            use_images = False
        elif stage_name=="test" and INFER_SKIP_IMAGE_DOWNLOAD:
            use_images = False
        else:
            img = make_doc_views_from_base64(context) or [load_image_resized(context)]
            img = [x for x in img if x is not None]
            if len(img)>0:
                images = img
                use_images = True

    if use_images:
        messages=[
            {"role":"system","content":[{"type":"text","text":SYSTEM_INST}]},
            {"role":"user","content":[*([{"type":"image"}]*len(images)), {"type":"text","text":user_text or ""}]}
        ]
    else:
        messages=[
            {"role":"system","content":[{"type":"text","text":SYSTEM_INST}]},
            {"role":"user","content":[{"type":"text","text":user_text or ""}]}
        ]

    prompt = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    enc = processor(text=prompt, images=(images if use_images else None), return_tensors="pt", padding=False)
    if torch.cuda.is_available(): enc = {k:v.to(model.device) for k,v in enc.items()}

    # per-task 토큰/스톱
    start_pos = enc["input_ids"].shape[-1]
    sc = StoppingCriteriaList()
    if stage_name == "eval":
        if routed == "captioning":
            local_max = CAPTION_MAX_NEW_TOKENS_EVAL
            sc.append(StopOnSentenceCount(processor.tokenizer, start_pos, CAPTION_MAX_SENTENCES))
        elif routed == "summarization":
            local_max = SUM_MAX_NEW_TOKENS_EVAL
            sc.append(StopOnSentenceCount(processor.tokenizer, start_pos, SUM_MAX_SENTENCES))
        elif routed == "math":
            local_max = MATH_MAX_NEW_TOKENS_EVAL
        else:
            local_max = MAX_NEW_TOKENS_EVAL
    else:  # test
        if routed == "captioning":
            local_max = CAPTION_MAX_NEW_TOKENS_TEST
            sc.append(StopOnSentenceCount(processor.tokenizer, start_pos, CAPTION_MAX_SENTENCES))
        elif routed == "summarization":
            local_max = SUM_MAX_NEW_TOKENS_TEST
            sc.append(StopOnSentenceCount(processor.tokenizer, start_pos, SUM_MAX_SENTENCES))
        elif routed == "math":
            local_max = MATH_MAX_NEW_TOKENS_TEST
        else:
            local_max = MAX_NEW_TOKENS_TEST

    gen_ids = _generate_once_greedy(
        model, enc, local_max,
        stopping_criteria=sc,
        eos_token_id=processor.tokenizer.eos_token_id,
        pad_token_id=processor.tokenizer.pad_token_id
    )
    pred = _decode_new_tokens(processor, gen_ids, enc)
    return postprocess_output(pred, question=question, input_type=input_type, image_loaded=use_images, context=context)

# -----------------------------
# 콜백: 간이 평가(라우팅 기준) + ES (빠른 모드)
# -----------------------------
class MetricEvalEarlyStopCallback(TrainerCallback):
    def __init__(self, processor, eval_df, final_dir,
                 every_steps=EVAL_EVERY_STEPS, patience=ES_PATIENCE, min_delta=ES_MIN_DELTA):
        self.processor = processor
        self.eval_df   = eval_df
        self.final_dir = final_dir
        self.every_steps = every_steps
        self.patience = patience
        self.min_delta = min_delta
        self.best = None
        self.bad_cnt = 0

    @torch.no_grad()
    def _evaluate_small(self, model):
        if len(self.eval_df)==0: return 0.0, {}
        model.eval()
        per_task_scores = {}

        for _, row in self.eval_df.iterrows():
            ref  = str(row.get("output",""))
            tdet = determine_task(row.get("input_type",""), row.get("input",""), row.get("question",""))
            try:
                pred = _fast_generate_row(model, self.processor, row, "eval")
            except Exception:
                pred = "-"
            sdict = score_sample_routed(tdet, pred, ref)
            s     = collapse_task_metric(tdet, sdict)
            per_task_scores.setdefault(tdet, []).append(float(s))

        task_avgs = {t: (sum(v)/len(v) if len(v)>0 else 0.0) for t,v in per_task_scores.items()}
        macro = sum(task_avgs.values())/max(1, len(task_avgs))
        return macro, task_avgs

    def _maybe_eval_and_es(self, state, control, model, tag):
        if len(self.eval_df)==0: return control
        macro, task_avgs = self._evaluate_small(model)
        msg = f"[{tag}] step={state.global_step} macro={macro:.4f} | " + " ".join([f"{k}:{v:.3f}" for k,v in sorted(task_avgs.items())])
        print(msg)
        try:
            os.makedirs(self.final_dir, exist_ok=True)
            with open(os.path.join(self.final_dir, "last_eval_metrics.json"), "w") as f:
                json.dump({"step": int(state.global_step), "macro": float(macro), "by_task": {k: float(v) for k,v in task_avgs.items()}}, f)
        except Exception:
            pass

        improved = (self.best is None) or (macro > self.best + self.min_delta)
        if improved:
            self.best = macro; self.bad_cnt = 0
            try:
                os.makedirs(self.final_dir, exist_ok=True)
                model.save_pretrained(self.final_dir); self.processor.save_pretrained(self.final_dir)
                print(f"💾 새 베스트 저장 (macro={macro:.4f}) → {self.final_dir}")
            except Exception as e:
                print("베스트 저장 실패:", e)
            control.should_save = True
        else:
            self.bad_cnt += 1
            print(f"↳ 개선 없음 ({self.bad_cnt}/{self.patience})")
            if self.bad_cnt >= self.patience:
                print("🛑 Early Stop 발동 (patience 소진)")
                control.should_training_stop = True
        return control

    def on_step_end(self, args, state, control, **kwargs):
        model = kwargs.get("model", None)
        if model is None: return control
        if state.global_step>0 and (state.global_step % self.every_steps == 0):
            control = self._maybe_eval_and_es(state, control, model, tag="interval")
        return control

    def on_epoch_end(self, args, state, control, **kwargs):
        model = kwargs.get("model", None)
        control = self._maybe_eval_and_es(state, control, model, tag="epoch_end")
        return control

# -----------------------------
# 학습 파라미터 & 트레이너
# -----------------------------
os.makedirs(CKPT_DIR, exist_ok=True)

args = TrainingArguments(
    output_dir=CKPT_DIR,
    per_device_train_batch_size=PER_DEV_TRAIN_BS,
    per_device_eval_batch_size=PER_DEV_EVAL_BS,
    gradient_accumulation_steps=GRAD_ACC_STEPS,
    learning_rate=1e-4,
    num_train_epochs=1,
    bf16=USE_BF16,
    fp16=USE_FP16,
    logging_steps=20,
    save_steps=400,                # 평가 주기와 보조
    save_total_limit=4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    optim="paged_adamw_8bit",
    gradient_checkpointing=True,
    report_to="none",
    remove_unused_columns=False,
    dataloader_num_workers=2,
    dataloader_pin_memory=True,
    save_safetensors=True,
    max_grad_norm=MAX_GRAD_NORM,
)

# 평가 서브셋 구축 (속도↑)
eval_subset_df = build_eval_subset(df_va, per_task=EVAL_SAMPLES_PER_TASK, max_total=EVAL_MAX_TOTAL_SAMPLES, seed=SUBSET_SEED)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,  # (HF 트레이너용, 실제 평가는 커스텀 콜백에서 eval_subset_df 사용)
    data_collator=collate,
    callbacks=[
        TimeLimitCallback(max_seconds=int(LIMIT_HOURS*3600)),
        MetricEvalEarlyStopCallback(
            processor=processor,
            eval_df=eval_subset_df,
            final_dir=FINAL_DIR,
            every_steps=EVAL_EVERY_STEPS,
            patience=ES_PATIENCE,
            min_delta=ES_MIN_DELTA,
        ),
    ],
)

# -----------------------------
# Torch<2.6 CVE 우회: weights-only resume
# -----------------------------
def try_load_adapter_weights_from(path):
    st_path = os.path.join(path, "adapter_model.safetensors")
    if not os.path.exists(st_path):
        return False
    try:
        sd = safe_load_file(st_path)
        missing, unexpected = model.load_state_dict(sd, strict=False)
        print(f"weights-only resume: missing={len(missing)}, unexpected={len(unexpected)}")
        return True
    except Exception as e:
        print("weights-only load 실패:", e)
        return False

def torch_version_tuple():
    ver = torch.__version__.split('+')[0]
    return tuple(int(x) for x in ver.split('.')[:3])

last_ckpt = get_last_checkpoint(CKPT_DIR) if os.path.isdir(CKPT_DIR) else None
can_secure_resume = torch_version_tuple() >= (2,6,0)

if last_ckpt and not can_secure_resume:
    print(f"🔁 Torch<{2.6}: optimizer 복원 생략, 가중치만 이어서 학습 ({last_ckpt})")
    ok = try_load_adapter_weights_from(last_ckpt)
    if not ok and os.path.isdir(FINAL_DIR):
        print("체크포인트 로드 실패 → FINAL_DIR에서 가중치 복원 시도")
        ok = try_load_adapter_weights_from(FINAL_DIR)
    last_ckpt = None
elif last_ckpt:
    print(f"🔁 체크포인트에서 재개: {last_ckpt}")
else:
    print("🚀 새 학습 시작")

# -----------------------------
# 학습 & 저장
# -----------------------------
try:
    if RUN_TRAIN:
        trainer.train(resume_from_checkpoint=last_ckpt)
    else:
        print("🚫 RUN_TRAIN=False: 학습을 건너뜁니다. (바로 추론 단계로)")
except Exception as e:
    print(f"⚠️ 학습 중 예외 발생: {e}\n현재 상태 저장을 시도합니다.")
finally:
    os.makedirs(FINAL_DIR, exist_ok=True)
    try:
        model.save_pretrained(FINAL_DIR)
        processor.save_pretrained(FINAL_DIR)
    except Exception as e:
        print("최종 저장 중 경고:", e)
    print("모델/프로세서 저장 완료 →", FINAL_DIR)
    torch.cuda.empty_cache(); gc.collect()

# -----------------------------
# 테스트셋 추론 → submission.csv (부분 저장/재개)
# -----------------------------
print("Generating submission.csv from test set...")
model.eval(); torch.set_grad_enabled(False)
df_test = safe_read_df(CSV_TEST).reset_index(drop=True)

def build_messages_for_infer(row):
    input_type = row.get("input_type","")
    question   = row.get("question","")
    context    = row.get("input","")
    user_text  = build_text_prompt(input_type, context, question)
    routed     = determine_task(input_type, context, question)

    use_images = False
    images = None
    if input_type=="image":
        if INFER_SKIP_IMAGE_DOWNLOAD:
            use_images = False
        else:
            img = make_doc_views_from_base64(context) or [load_image_resized(context)]
            img = [x for x in img if x is not None]
            if len(img)>0:
                images = img
                use_images = True

    if use_images:
        messages = [
            {"role":"system","content":[{"type":"text","text":SYSTEM_INST}]},
            {"role":"user","content":[*([{"type":"image"}]*len(images)), {"type":"text","text":user_text or ""}]}
        ]; return messages, images
    else:
        messages = [
            {"role":"system","content":[{"type":"text","text":SYSTEM_INST}]},
            {"role":"user","content":[{"type":"text","text":user_text or ""}]}
        ]; return messages, None

# 부분 저장 이어받기
processed={}
if os.path.exists(SUBMIT_PART):
    try:
        dfp=pd.read_csv(SUBMIT_PART)
        for _, r in dfp.iterrows(): processed[int(r["id"])]=str(r["output"])
        print(f"🔁 기존 부분 결과 로드: {len(processed)}개")
    except Exception as e:
        print("부분 결과 로드 실패, 새로 시작:", e)

preds_map=dict(processed)
elapsed_total=time.time()-GLOBAL_START
limit_sec=max(0,int(LIMIT_HOURS*3600)-int(elapsed_total))
infer_start=time.time()
def time_left_ok(): return (time.time()-infer_start) < max(0, limit_sec-5*60)

todo_ids=[i for i in range(len(df_test)) if i not in preds_map]
for idx, i in enumerate(todo_ids, 1):
    if not time_left_ok():
        print("⏰ 추론 시간 한계 도달. 부분 결과 저장 후 종료."); break
    row=df_test.iloc[i]
    try:
        messages, images = build_messages_for_infer(row)
        prompt = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        enc = processor(text=prompt, images=images, return_tensors="pt", padding=False)
        if torch.cuda.is_available(): enc = {k:v.to(model.device) for k,v in enc.items()}

        tdet = determine_task(row.get("input_type",""), row.get("input",""), row.get("question",""))

        # per-task 토큰/스톱
        start_pos = enc["input_ids"].shape[-1]
        sc = StoppingCriteriaList()
        if tdet == "captioning":
            local_max = CAPTION_MAX_NEW_TOKENS_TEST
            sc.append(StopOnSentenceCount(processor.tokenizer, start_pos, CAPTION_MAX_SENTENCES))
        elif tdet == "summarization":
            local_max = SUM_MAX_NEW_TOKENS_TEST
            sc.append(StopOnSentenceCount(processor.tokenizer, start_pos, SUM_MAX_SENTENCES))
        elif tdet == "math":
            local_max = MATH_MAX_NEW_TOKENS_TEST
        else:
            local_max = MAX_NEW_TOKENS_TEST

        # 추론도 greedy + 스톱 기준
        gen_ids = _generate_once_greedy(
            model, enc, local_max,
            stopping_criteria=sc,
            eos_token_id=processor.tokenizer.eos_token_id,
            pad_token_id=processor.tokenizer.pad_token_id
        )
        pred = _decode_new_tokens(processor, gen_ids, enc)
        pred = postprocess_output(pred,
                                  question=row.get("question",""),
                                  input_type=row.get("input_type",""),
                                  image_loaded=(images is not None),
                                  context=row.get("input",""))
        if not pred.strip():
            pred = "-"
    except Exception:
        pred = "-"

    preds_map[i] = pred

    if (idx % INFER_SAVE_EVERY == 0) or (idx == len(todo_ids)):
        tmp = pd.DataFrame({"id": sorted(preds_map.keys()),
                            "output": [preds_map[k] for k in sorted(preds_map.keys())]})
        tmp["output"] = tmp["output"].apply(lambda x: x if str(x).strip() else "-")
        tmp.to_csv(SUBMIT_PART, index=False, encoding="utf-8")
        print(f"[{idx}/{len(todo_ids)}] 부분 저장 → {SUBMIT_PART}")

all_ids = list(range(len(df_test)))
outputs = [preds_map.get(i, "-") for i in all_ids]
outputs = [o if str(o).strip() else "-" for o in outputs]
pd.DataFrame({"id": all_ids, "output": outputs}).to_csv(SUBMIT_OUT, index=False, encoding="utf-8")
print("✅ submission.csv saved to:", SUBMIT_OUT)
print("✅ 부분 저장본(백업):", SUBMIT_PART)
